# Imports

In [30]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from tqdm import tqdm
import math

from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from copy import deepcopy

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.base import clone

from scipy.spatial.distance import cdist
from scipy.stats import ks_2samp
from scipy.optimize import minimize
from scipy.stats import wasserstein_distance

from sklearn.metrics import average_precision_score


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

y = np.where(y == 11, 0, 1)

# Setup

In [33]:
def create_phi(normal_data, c):
    """
    Concept creation function for normal data.
    Uses k-Means clustering to partition normal data into c clusters.
    
    Args:
        normal_data (numpy array): The normal data points.
        c (int): Number of desired normal concepts.
    
    Returns:
        list of numpy arrays: List of normal clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(normal_data)
    
    normal_concepts = [normal_data[labels == i] for i in range(c)]
    print("Finished creating normal concepts")
    
    return normal_concepts


def create_gamma(anomaly_data, c):
    """
    Concept creation function for anomaly data.
    Uses k-Means clustering to partition anomaly data into c clusters.
    
    Args:
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired anomaly concepts.
    
    Returns:
        list of numpy arrays: List of anomaly clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(anomaly_data)
    
    anomaly_concepts = [anomaly_data[labels == i] for i in range(c)]
    print("Finished creating anomaly concepts")
    
    return anomaly_concepts
    
def match_lambda(anomaly_concepts, normal_concepts):
    """
    Matches each normal concept with the closest anomaly concept.
    Uses Euclidean distance to determine the best match.
    
    Args:
        anomaly_concepts (list of numpy arrays): List of anomaly clusters.
        normal_concepts (list of numpy arrays): List of normal clusters.
    
    Returns:
        list of tuples: Pairs of (normal_concept, matched_anomaly_concept)
    """
    pairs = []
    remaining_anomalies = anomaly_concepts.copy()

    for normal_concept in normal_concepts:
        normal_centroid = np.mean(normal_concept, axis=0)
        anomaly_centroids = [np.mean(ac, axis=0) for ac in remaining_anomalies]

        distances = cdist([normal_centroid], anomaly_centroids, metric='euclidean')[0]
        closest_idx = np.argmin(distances)

        pairs.append((normal_concept, remaining_anomalies[closest_idx]))
        remaining_anomalies.pop(closest_idx)

    print("Finished matching concept pairs")
    
    return pairs

def lifelong_roc_auc(R):
    """
    Computes the Lifelong ROC-AUC metric.
    
    Args:
        R (numpy array): NxN matrix of ROC-AUC scores, where R[i, j] is the model's 
                         performance on concept j after learning concept i.
    
    Returns:
        float: Lifelong ROC-AUC score.
    """
    N = R.shape[0]
    lower_triangular_sum = np.sum(np.tril(R))
    normalization_factor = (N * (N + 1)) / 2

    return lower_triangular_sum / normalization_factor

def BWT(R):
    """
    Computes the Backward Transfer (BWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: BWT score.
    """
    N = R.shape[0]
    backward_transfer = 0
    count = 0

    for i in range(1, N):
        for j in range(i):
            backward_transfer += (R[i, j] - R[j, j])
            count += 1

    return backward_transfer / count if count > 0 else 0

def FWT(R):
    """
    Computes the Forward Transfer (FWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: FWT score.
    """
    N = R.shape[0]
    forward_transfer = 0
    count = 0

    for i in range(N):
        for j in range(i + 1, N): 
            forward_transfer += R[i, j]
            count += 1

    return forward_transfer / count if count > 0 else 0 

def kolmogorov_smirnov_test(X_old, X_new, alpha=0.05):
    """Detect concept drift using KS-test on feature distributions."""
    
    p_values = [ks_2samp(X_old[:, i], X_new[:, i]).pvalue for i in range(X_old.shape[1])]
    return np.any(np.array(p_values) < alpha)

def histogram_binning(X, bins=25):
    """Convert sample distributions into histograms."""
    
    return np.array([np.histogram(X[:, i], bins=bins, density=True)[0] for i in range(X.shape[1])]).T

def kl_divergence(P, Q):
    """Compute KL divergence between two distributions."""
    
    P, Q = np.clip(P, 1e-10, None), np.clip(Q, 1e-10, None)  # Avoid log(0)
    return np.sum(P * np.log(P / Q))

def strategic_sample_selection(X_old, X_new, top_k=100, learning_rate=0.01, num_iterations=100):
    """
    Selects representative new samples by minimizing KL divergence.
    
    Args:
        X_old (numpy.ndarray): Old memory buffer samples.
        X_new (numpy.ndarray): Incoming new samples.
        top_k (int): Number of samples to retain.
        learning_rate (float): Step size for optimization.
        num_iterations (int): Number of optimization steps.

    Returns:
        numpy.ndarray: Selected representative new samples.
    """
    
    H_old, H_new = histogram_binning(X_old), histogram_binning(X_new)
    m_n = np.random.rand(H_new.shape[0])  

    def loss_function(m_n):
        """Computes KL divergence loss for selected samples."""
        weighted_H_new = H_new * m_n[:, np.newaxis]  
        combined_H = (H_old + weighted_H_new) / 2 
        return kl_divergence(H_new, combined_H) 

    progress_bar = tqdm(total=num_iterations, desc="Optimizing Sample Selection", position=0, leave=True)

    def callback(xk):
        progress_bar.update(1)  

    result = minimize(loss_function, m_n, method="L-BFGS-B", bounds=[(0, 1)] * len(m_n), 
                      options={"maxiter": num_iterations, "ftol": 1e-10}, callback=callback)

    progress_bar.close()

    selected_indices = np.argsort(result.x)[-top_k:]

    return X_new[selected_indices] 


def update_memory_buffer(X_old, X_new_selected, memory_size=3000):
    """Updates memory buffer using strategic forgetting."""
    updated_buffer = np.vstack((X_old, X_new_selected))  

    if updated_buffer.shape[0] > memory_size:
        updated_buffer = updated_buffer[-memory_size:]

    return updated_buffer

class HierarchicalMemory:
    def __init__(self, memory_limit=5000, pyramid_factor=2, centroids_per_concept=10):
        self.memory_limit = memory_limit
        self.pyramid_factor = pyramid_factor
        self.centroids_per_concept = centroids_per_concept
        self.memory = {}  # level: [concept1, concept2, ...]

    def add_concept(self, data, level=1):
        if level not in self.memory:
            self.memory[level] = []
        self.memory[level].append(np.array(data))
        self._summarize_memory()

    def _pyramidal_allocation(self):
        levels = sorted(self.memory.keys())
        weights = np.array([1 / (self.pyramid_factor ** (lvl - 1)) for lvl in levels])
        total_weight = weights.sum()
        allocations = (weights / total_weight) * self.memory_limit
        return {lvl: int(alloc) for lvl, alloc in zip(levels, allocations)}

    def _summarize_concept(self, concept, n_samples):
        if len(concept) <= n_samples:
            return concept
        kmeans = KMeans(n_clusters=min(self.centroids_per_concept, len(concept)), random_state=42).fit(concept)
        centroids = kmeans.cluster_centers_
        distances = np.linalg.norm(concept[:, None] - centroids, axis=2)
        closest_indices = np.argmin(distances, axis=0)
        summarized = concept[closest_indices]
        return summarized

    def _summarize_memory(self):
        allocations = self._pyramidal_allocation()
        for level, concepts in self.memory.items():
            summarized_level = []
            alloc_per_concept = max(1, allocations[level] // len(concepts))
            for concept in concepts:
                summarized = self._summarize_concept(concept, alloc_per_concept)
                summarized_level.append(summarized)
            self.memory[level] = summarized_level

    def get_all_memory(self):
        all_data = []
        for level_concepts in self.memory.values():
            for concept in level_concepts:
                all_data.append(concept)
        return np.vstack(all_data) if all_data else np.empty((0,))

def scenario_design(normal_data, anomaly_data, c):
    """
    Implements Algorithm 1 to create a lifelong learning scenario.
    
    Args:
        normal_data (numpy array): The normal data points.
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired concepts.
    
    Returns:
        list of tuples: List of (normal_concept, anomaly_concept) pairs forming the scenario.
    """
    normal_concepts = create_phi(normal_data, c)
    anomaly_concepts = create_gamma(anomaly_data, c)
    
    scenario = match_lambda(anomaly_concepts, normal_concepts)
    
    return scenario

def evaluation_protocol(T, E, Y, model, strategy="naive", replay_buffer_size=5000, memory_size=5000, alpha=0.05):
    """
    Implements Algorithm 2: Lifelong Learning Evaluation Protocol with multiple strategies.
    
    Args:
        T (list): Sequence of N training sets.
        E (list): Sequence of N testing sets.
        Y (list): Sequence of true labels for test sets.
        model (sklearn.base.BaseEstimator): A scikit-learn-like model instance that supports `fit` and `decision_function`.
        strategy (str): Strategy for training.
        replay_buffer_size (int): Maximum size of replay buffer if applicable
        memory_size (int): Maximum memory size if applicable
        alpha (float): KS-test threshold for drift detection.

    Returns:
        numpy array: NxN results matrix R where R[i, j] is ROC-AUC of model on E[j] after learning T[i].
    """
    N = len(T)
    R = np.zeros((N, N))  

    if strategy in ["cumulative"]:
        cumulative_data = []
    
    if strategy in ["replay"]:
        replay_buffer = []

    if strategy == "SSF":
        memory_buffer = None 

    if strategy == "hierarchical":
        h_memory = HierarchicalMemory(memory_limit=memory_size, pyramid_factor=2, centroids_per_concept=10)

    
    inference_times = []
    for i, Ti in tqdm(enumerate(T), desc=f"Evaluating using {strategy} strategy"):
        current_model = deepcopy(model)

        # -- NAIVE --
        if strategy == "naive":
            current_model.fit(Ti)

        # -- CUMULATIVE --
        elif strategy == "cumulative":
            cumulative_data.extend(Ti.tolist())
            current_model.fit(np.array(cumulative_data)) 

        # -- REPLAY -- 
        elif strategy == "replay":
            if replay_buffer:
                combined_data = np.vstack((np.array(replay_buffer), Ti))
            else:
                combined_data = Ti

            current_model.fit(combined_data)
            replay_buffer.extend(Ti.tolist())

            if len(replay_buffer) > replay_buffer_size:
                replay_buffer = replay_buffer[-replay_buffer_size:]
        
        # -- SSF -- 
        elif strategy == "SSF":
            if memory_buffer is None:
                memory_buffer = Ti[:memory_size]  
            else:
                drift_detected = kolmogorov_smirnov_test(memory_buffer, Ti, alpha)
                if drift_detected:
                    X_new_selected = strategic_sample_selection(memory_buffer, Ti, top_k=1000)
                    memory_buffer = update_memory_buffer(memory_buffer, X_new_selected, memory_size=memory_size)
            memory_buffer = np.unique(memory_buffer, axis=0)
            current_model.fit(memory_buffer)

        # -- HIERARCHICAL --
        elif strategy == "hierarchical":

            memory_data = h_memory.get_all_memory()
            if memory_data.size == 0:
                drift_level = 1
            else:
                drift_distances = [
                    wasserstein_distance(Ti[:, d], memory_data[:, d])
                    for d in range(Ti.shape[1])
                ]
                drift_score = np.mean(drift_distances)
                print(f"drift: {drift_score}")
                
                if drift_score < 0.05:
                    drift_level = 1
                elif drift_score < 0.1:
                    drift_level = 2
                elif drift_score < 0.2:
                    drift_level = 3
                else:
                    drift_level = 4
        
            h_memory.add_concept(Ti, level=drift_level)
            summarized_memory = h_memory.get_all_memory()
            current_model.fit(summarized_memory)

        # -- Evaluation --
        for j, ((Ej_normal, Ej_anomaly), (y_normal, y_anomaly)) in enumerate(zip(E, Y)):
            test_data = np.vstack((Ej_normal, Ej_anomaly))
            test_labels = np.hstack((y_normal, y_anomaly))  

            start_time = time.time()
            scores = -current_model.decision_function(test_data)  
            elapsed = time.time() - start_time
            inference_times.extend([elapsed / len(test_data)] * len(test_data))
            
            R[i, j] = average_precision_score(test_labels, scores)

    avg_inference_time = np.mean(inference_times)
    print(f"Average Inference Time per Evaluation: {avg_inference_time:.10f} seconds")

    return R


# Experiments

In [34]:
num_concepts = 5

X_normal = X[y == 0]  
X_anomaly = X[y == 1]

normal_concepts = create_phi(X_normal, num_concepts)
anomaly_concepts = create_gamma(X_anomaly, num_concepts)

concept_pairs = match_lambda(anomaly_concepts, normal_concepts)

T = []  
E = [] 
Y = []

for normal, anomaly in concept_pairs:

    normal_train, normal_test = train_test_split(normal, test_size=0.3, random_state=42)
    anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.3, random_state=42)  

    T.append(normal_train)
    E.append((normal_test, anomaly_test))

    y_normal_test = np.zeros(len(normal_test))
    y_anomaly_test = np.ones(len(anomaly_test))
    
    Y.append((y_normal_test, y_anomaly_test))

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs


# Eval

## LOF

In [19]:
R_hm = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_lof.py:283: UserWarning: n_neighbors (20) is greater than the total number of samples (10). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
Evaluating using hierarchical strategy: 1it [00:00,  2.57it/s]

drift: 0.15575845828560167


Evaluating using hierarchical strategy: 2it [00:01,  1.17it/s]

drift: 0.10687482627406424


Evaluating using hierarchical strategy: 3it [00:02,  1.42it/s]

drift: 0.08550491766410032


Evaluating using hierarchical strategy: 4it [00:02,  1.31it/s]

drift: 0.14433198880791898


Evaluating using hierarchical strategy: 5it [00:03,  1.43it/s]

Lifelong ROC-AUC: 0.8010632635023135, BWT: -0.03189907514987032, FWT: 0.61397202689583


In [26]:
R_ssf = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 273.05it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 256.08it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 276.87it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 326.53it/s]

Evaluating using SSF strategy: 5it [00:05,  1.14s/it]

Lifelong ROC-AUC: 0.7544561432893502, BWT: -0.0039135441339650965, FWT: 0.45301001764958054


In [12]:
R_naive = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:16,  3.27s/it]

Lifelong PR-AUC: 0.6259146979625797, BWT: -0.2298943824165936, FWT: 0.42150088732223623


In [7]:
R_cumulative = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [02:10, 26.08s/it]

Lifelong ROC-AUC: 0.9238354062988797, BWT: -0.004499087364359666, FWT: 0.17379610432039533


In [19]:
R_replay = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:19,  3.87s/it]

Average Inference Time per Evaluation: 0.0000153946 seconds
Lifelong PR-AUC: 0.7076565525940482, BWT: -0.1496000260952163, FWT: 0.3201269420302635


## IF

In [20]:
R_hm = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 1it [00:00,  1.35it/s]

drift: 0.15575845828560167


Evaluating using hierarchical strategy: 2it [00:02,  1.24s/it]

drift: 0.10687482627406424


Evaluating using hierarchical strategy: 3it [00:03,  1.11s/it]

drift: 0.08550491766410032


Evaluating using hierarchical strategy: 4it [00:04,  1.26s/it]

drift: 0.14433198880791898


Evaluating using hierarchical strategy: 5it [00:05,  1.16s/it]

Lifelong ROC-AUC: 0.6850161874595665, BWT: -0.034863433306860495, FWT: 0.6161049330175039


In [27]:
R_ssf = evaluation_protocol(T, E, Y,  IsolationForest(n_estimators=100), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 281.14it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 253.28it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 237.31it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 244.61it/s]

Evaluating using SSF strategy: 5it [00:05,  1.06s/it]

Lifelong ROC-AUC: 0.6559285630296237, BWT: 0.06868127809581366, FWT: 0.804018261100736


In [14]:
R_naive = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:03,  1.38it/s]

Lifelong PR-AUC: 0.6032600619774786, BWT: -0.1718058652463209, FWT: 0.6548393577005718


In [9]:
R_cumulative = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [00:07,  1.55s/it]

Lifelong ROC-AUC: 0.753897896406868, BWT: -0.0066406078135951676, FWT: 0.7719174809547474


In [20]:
R_replay = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:04,  1.10it/s]

Average Inference Time per Evaluation: 0.0000027950 seconds
Lifelong PR-AUC: 0.623437461532319, BWT: -0.15620535333182275, FWT: 0.6756342504106363


## SGDOCSVM

In [21]:
R_hm = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 1it [00:00,  3.21it/s]

drift: 0.15575845828560167


Evaluating using hierarchical strategy: 3it [00:01,  1.91it/s]

drift: 0.10687482627406424
drift: 0.08550491766410032


Evaluating using hierarchical strategy: 5it [00:02,  1.83it/s]

drift: 0.14433198880791898
Lifelong ROC-AUC: 0.6133161578664278, BWT: -0.1228301210171924, FWT: 0.5027924783558533


In [28]:
R_ssf = evaluation_protocol(T, E, Y,  SGDOneClassSVM(), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 288.13it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 307.55it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 306.85it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 271.76it/s]

Evaluating using SSF strategy: 5it [00:02,  2.14it/s]

Lifelong ROC-AUC: 0.790790866239933, BWT: -0.004091332504534584, FWT: 0.5467376505389003


In [16]:
R_naive = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:00,  8.77it/s]

Lifelong PR-AUC: 0.6338207341822814, BWT: -0.15638596219766385, FWT: 0.5669742953319241


In [17]:
R_cumulative = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [00:04,  1.03it/s]


Lifelong ROC-AUC: 0.597799790906325, BWT: -0.215598947769658, FWT: 0.6579349113663381


In [21]:
R_replay = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:01,  2.77it/s]

Average Inference Time per Evaluation: 0.0000001217 seconds
Lifelong PR-AUC: 0.6296275394432779, BWT: -0.15245286514912806, FWT: 0.5503971510496488


# SLAD

In [22]:
from deepod.models.tabular import SLAD

In [8]:
R_hm = evaluation_protocol(T, E, Y, SLAD(), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  9 10 11 12 13 14 16 17 18 19 20 22 23 24 25 26 27
 28 30 31 32 33 34 35 36 37 38 40 41 42 44 45 46 47 48 49 50 51 53 54 55
 56 57]
epoch  1, training loss: 0.789064, time: 136.0s
epoch 10, training loss: 0.745528, time: 0.0s
epoch 20, training loss: 0.716392, time: 0.0s
epoch 30, training loss: 0.690674, time: 0.0s
epoch 40, training loss: 0.671540, time: 0.0s
epoch 50, training loss: 0.664055, time: 0.0s
epoch 60, training loss: 0.661995, time: 0.0s
epoch 70, training loss: 0.654417, time: 0.0s
epoch 80, training loss: 0.654082, time: 0.0s
epoch 90, training loss: 0.652842, time: 0.0s
epoch100, training loss: 0.648777, time: 0.0s
Start Inference on the training data...


Evaluating using hierarchical strategy: 1it [05:54, 354.02s/it]

drift: 0.15575845828560167
Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  9 10 11 12 13 15 16 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 37 38 39 40 41 42 43 45 46 48 49 50 51 52 53 54 55 56
 57 58]
epoch  1, training loss: 0.821572, time: 0.0s
epoch 10, training loss: 0.729630, time: 0.0s
epoch 20, training loss: 0.705427, time: 0.0s
epoch 30, training loss: 0.697949, time: 0.0s
epoch 40, training loss: 0.695073, time: 0.0s
epoch 50, training loss: 0.693725, time: 0.0s
epoch 60, training loss: 0.693104, time: 0.0s
epoch 70, training loss: 0.692327, time: 0.0s
epoch 80, training loss: 0.692073, time: 0.0s
epoch 90, training loss: 0.692050, time: 0.0s
epoch100, training loss: 0.691381, time: 0.0s
Start Inference on the training data...


Evaluating using hierarchical strategy: 2it [07:04, 187.13s/it]

...

drift: 0.08550491766410032
Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  7  8  9 10 12 13 14 15 16 17 18 19 21 22 23 24 25 26 27
 29 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54
 56 58]
epoch  1, training loss: 0.792425, time: 0.0s
epoch 10, training loss: 0.687732, time: 0.1s
epoch 20, training loss: 0.678437, time: 0.0s
epoch 30, training loss: 0.675762, time: 0.0s
epoch 40, training loss: 0.674444, time: 0.0s
epoch 50, training loss: 0.673544, time: 0.0s
epoch 60, training loss: 0.674035, time: 0.0s
epoch 70, training loss: 0.673031, time: 0.0s
epoch 80, training loss: 0.673678, time: 0.0s
epoch 90, training loss: 0.673214, time: 0.0s
epoch100, training loss: 0.672836, time: 0.0s
Start Inference on the training data...


Evaluating using hierarchical strategy: 4it [09:28, 109.66s/it]

drift: 0.14433198880791898
Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  5  7  8  9 10 12 13 14 15 16 18 19 20 21 22 23 24 25 27 28 29
 30 31 33 34 35 36 37 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54 55
 57 58]
epoch  1, training loss: 0.771024, time: 0.0s
epoch 10, training loss: 0.658865, time: 0.0s
epoch 20, training loss: 0.647916, time: 0.0s
epoch 30, training loss: 0.643690, time: 0.0s
epoch 40, training loss: 0.642270, time: 0.0s
epoch 50, training loss: 0.641485, time: 0.0s
epoch 60, training loss: 0.641479, time: 0.0s
epoch 70, training loss: 0.641318, time: 0.0s
epoch 80, training loss: 0.640435, time: 0.0s
epoch 90, training loss: 0.639958, time: 0.0s
epoch100, training loss: 0.640519, time: 0.0s
Start Inference on the training data...


Evaluating using hierarchical strategy: 5it [10:37, 127.52s/it]

Lifelong ROC-AUC: 0.16119022197491426, BWT: 0.055047207586719736, FWT: 0.22260804538321585


In [10]:
R_ssf = evaluation_protocol(T, E, Y, SLAD(), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  9 10 11 12 13 14 16 17 18 19 20 22 23 24 25 26 27
 28 30 31 32 33 34 35 36 37 38 40 41 42 44 45 46 47 48 49 50 51 53 54 55
 56 57]
epoch  1, training loss: 0.689123, time: 0.9s
epoch 10, training loss: 0.610598, time: 0.9s
epoch 20, training loss: 0.610824, time: 0.9s
epoch 30, training loss: 0.610099, time: 0.9s
epoch 40, training loss: 0.609887, time: 0.9s
epoch 50, training loss: 0.609973, time: 0.9s
epoch 60, training loss: 0.609928, time: 0.2s
epoch 70, training loss: 0.610051, time: 0.2s
epoch 80, training loss: 0.610074, time: 0.2s
epoch 90, training loss: 0.610767, time: 0.2s
epoch100, training loss: 0.609855, time: 0.2s
Start Inference on the training data...


Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 242.42it/s]


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 2  3  4  5  6  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24 26 27 28
 30 31 32 33 34 35 36 37 38 39 40 42 43 44 46 47 48 49 50 51 52 53 54 55
 56 57]
epoch  1, training loss: 0.683285, time: 0.9s
epoch 10, training loss: 0.606829, time: 0.9s
epoch 20, training loss: 0.606917, time: 0.2s
epoch 30, training loss: 0.606242, time: 0.2s
epoch 40, training loss: 0.605342, time: 0.2s
epoch 50, training loss: 0.606010, time: 0.2s
epoch 60, training loss: 0.605944, time: 0.2s
epoch 70, training loss: 0.605754, time: 0.2s
epoch 80, training loss: 0.605357, time: 0.2s
epoch 90, training loss: 0.605265, time: 0.3s
epoch100, training loss: 0.606159, time: 0.2s
Start Inference on the training data...


Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 249.32it/s]


...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 21 22 23 25 26 27
 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 50 51 53 55
 56 58]
epoch  1, training loss: 0.713907, time: 0.9s
epoch 10, training loss: 0.641691, time: 0.3s
epoch 20, training loss: 0.639720, time: 0.2s
epoch 30, training loss: 0.639970, time: 0.9s
epoch 40, training loss: 0.639926, time: 0.9s
epoch 50, training loss: 0.640661, time: 0.9s
epoch 60, training loss: 0.640567, time: 0.2s
epoch 70, training loss: 0.640408, time: 0.2s
epoch 80, training loss: 0.640357, time: 0.9s
epoch 90, training loss: 0.639724, time: 0.2s
epoch100, training loss: 0.640057, time: 0.2s
Start Inference on the training data...


Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 288.45it/s]


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 24 25 28 29 30
 31 32 33 34 35 36 37 38 39 40 41 43 44 45 46 47 48 49 50 51 52 53 54 55
 57 58]
epoch  1, training loss: 0.682585, time: 0.9s
epoch 10, training loss: 0.606542, time: 0.9s
epoch 20, training loss: 0.606012, time: 0.9s
epoch 30, training loss: 0.606196, time: 0.2s
epoch 40, training loss: 0.605747, time: 0.2s
epoch 50, training loss: 0.605817, time: 0.2s
epoch 60, training loss: 0.605292, time: 0.2s
epoch 70, training loss: 0.605695, time: 0.2s
epoch 80, training loss: 0.606173, time: 0.2s
epoch 90, training loss: 0.605267, time: 0.2s
epoch100, training loss: 0.605704, time: 0.2s
Start Inference on the training data...


Evaluating using SSF strategy: 5it [23:46, 285.36s/it]

Lifelong ROC-AUC: 0.15203513147863595, BWT: 0.00557263662112336, FWT: 0.3739874334612322


In [19]:
R_naive = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  9 10 11 12 13 14 16 17 18 19 20 22 23 24 25 26 27
 28 30 31 32 33 34 35 36 37 38 40 41 42 44 45 46 47 48 49 50 51 53 54 55
 56 57]
epoch  1, training loss: 0.651251, time: 131.3s
epoch 10, training loss: 0.623366, time: 0.7s
Start Inference on the training data...


Evaluating using naive strategy: 1it [05:44, 344.49s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  9 10 11 12 13 15 16 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 37 38 39 40 41 42 43 45 46 48 49 50 51 52 53 54 55 56
 57 58]
epoch  1, training loss: 0.655849, time: 3.3s
epoch 10, training loss: 0.648188, time: 3.3s
Start Inference on the training data...


Evaluating using naive strategy: 2it [07:49, 215.65s/it]

...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  7  8  9 10 12 13 14 15 16 17 18 19 21 22 23 24 25 26 27
 29 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54
 56 58]
epoch  1, training loss: 0.650727, time: 2.5s
epoch 10, training loss: 0.641749, time: 2.2s
Start Inference on the training data...


Evaluating using naive strategy: 4it [10:49, 132.67s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  5  7  8  9 10 12 13 14 15 16 18 19 20 21 22 23 24 25 27 28 29
 30 31 33 34 35 36 37 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54 55
 57 58]
epoch  1, training loss: 0.666014, time: 1.3s
epoch 10, training loss: 0.650179, time: 1.5s
Start Inference on the training data...


Evaluating using naive strategy: 5it [12:14, 146.90s/it]

Lifelong PR-AUC: 0.4402965545107119, BWT: 0.17382286318518067, FWT: 0.510222573327393


In [23]:
R_replay = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  9 10 11 12 13 14 16 17 18 19 20 22 23 24 25 26 27
 28 30 31 32 33 34 35 36 37 38 40 41 42 44 45 46 47 48 49 50 51 53 54 55
 56 57]
epoch  1, training loss: 0.651251, time: 0.9s
epoch 10, training loss: 0.623366, time: 0.7s
Start Inference on the training data...


Evaluating using replay strategy: 1it [01:27, 87.06s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  6  9 10 11 12 13 15 16 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 37 38 39 40 41 42 43 45 46 48 49 50 51 52 53 54 55 56
 57 58]
epoch  1, training loss: 0.632842, time: 3.4s
epoch 10, training loss: 0.624401, time: 3.4s
Start Inference on the training data...


Evaluating using replay strategy: 2it [03:49, 119.83s/it]

...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  4  5  7  8  9 10 12 13 14 15 16 17 18 19 21 22 23 24 25 26 27
 29 31 32 33 34 35 36 37 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54
 56 58]
epoch  1, training loss: 0.684525, time: 2.3s
epoch 10, training loss: 0.676974, time: 2.3s
Start Inference on the training data...


Evaluating using replay strategy: 4it [07:13, 109.03s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  2  3  5  7  8  9 10 12 13 14 15 16 18 19 20 21 22 23 24 25 27 28 29
 30 31 33 34 35 36 37 38 39 40 41 42 43 45 46 47 48 49 50 51 52 53 54 55
 57 58]
epoch  1, training loss: 0.662475, time: 1.4s
epoch 10, training loss: 0.648362, time: 1.4s
Start Inference on the training data...


Evaluating using replay strategy: 5it [08:50, 106.16s/it]

Average Inference Time per Evaluation: 0.0003795277 seconds
Lifelong PR-AUC: 0.38988992723596877, BWT: 0.1048683185396457, FWT: 0.4867854268343099


# ICL

In [24]:
from deepod.models.tabular import ICL

In [10]:
R_hm = evaluation_protocol(T, E, Y, ICL(epochs=100), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1/1 [00:00<00:00, 499.92it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 554.29it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 649.78it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 656.79it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 655.84it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 656.36it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 656.26it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 658.58it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 655.92it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 656.06it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 654.34it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 654.45it/s]
Evaluating using hierarchical strategy: 1it [00:17, 17.16s/it]

drift: 0.15575845828560167
Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features


testing: 100%|██████████| 1/1 [00:00<00:00, 522.33it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 549.21it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 652.87it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 656.23it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 654.90it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 654.63it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 653.86it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 653.44it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 653.78it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 655.12it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 653.67it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 654.09it/s]
Evaluating using hierarchical strategy: 2it [00:27, 13.33s/it]

...

drift: 0.08550491766410032
Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features


testing: 100%|██████████| 1/1 [00:00<00:00, 502.43it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 538.35it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 654.76it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 654.14it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 653.35it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 653.90it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 652.88it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 654.19it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 655.84it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 654.80it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 655.54it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 655.78it/s]
Evaluating using hierarchical strategy: 4it [00:48, 11.50s/it]

drift: 0.14433198880791898
Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features


testing: 100%|██████████| 1/1 [00:00<00:00, 537.46it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 548.71it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 655.77it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 657.21it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 656.11it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 652.81it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 654.51it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 654.93it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 655.40it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 656.23it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 654.82it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 655.34it/s]
Evaluating using hierarchical strategy: 5it [00:59, 11.84s/it]

Lifelong ROC-AUC: 0.16753311302147786, BWT: 0.053168243625636824, FWT: 0.32290299924103544


In [15]:
R_ssf = evaluation_protocol(T, E, Y, ICL(epochs=100), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 79/79 [00:00<00:00, 673.86it/s]

testing: 100%|██████████| 79/79 [00:00<00:00, 678.66it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 675.17it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 673.21it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 675.21it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 678.14it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 672.81it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 678.96it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 676.18it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 677.06it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 676.11it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 298.95it/s]


Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 79/79 [00:00<00:00, 677.20it/s]

testing: 100%|██████████| 79/79 [00:00<00:00, 680.49it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 678.82it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 677.45it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 677.85it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 677.43it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 678.29it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 674.15it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 678.45it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 678.08it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 678.96it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 292.33it/s]


...

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 79/79 [00:00<00:00, 683.06it/s]

testing: 100%|██████████| 79/79 [00:00<00:00, 679.62it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 679.83it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 680.88it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 679.45it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 680.67it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 672.05it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 680.46it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 676.72it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 678.48it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 678.89it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 312.29it/s]


Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 79/79 [00:00<00:00, 680.86it/s]

testing: 100%|██████████| 79/79 [00:00<00:00, 680.44it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 677.37it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 678.04it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 678.40it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 673.14it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 678.78it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 681.84it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 679.25it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 677.88it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 675.15it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 677.37it/s]
Evaluating using SSF strategy: 5it [05:30, 66.07s/it]

Lifelong ROC-AUC: 0.17998268086177635, BWT: 0.022856113012170224, FWT: 0.475937638473041


In [22]:
R_naive = evaluation_protocol(T, E, Y, ICL(epochs=100), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 312/312 [00:00<00:00, 625.25it/s]

testing: 100%|██████████| 312/312 [00:00<00:00, 630.25it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 629.01it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 629.95it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 629.51it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 630.33it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 628.90it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 629.13it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 628.15it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 630.71it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 629.61it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 628.20it/s]
Evaluating using naive strategy: 1it [04:25, 265.55s/it]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1472/1472 [00:02<00:00, 628.94it/s]

testing: 100%|██████████| 1472/1472 [00:02<00:00, 631.67it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 630.17it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 630.89it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 630.50it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 630.09it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 626.92it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 629.54it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 628.55it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 628.88it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 627.54it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 628.84it/s]
Evaluating using naive strategy: 2it [23:46, 792.28s/it]

...

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 968/968 [00:01<00:00, 628.09it/s]

testing: 100%|██████████| 968/968 [00:01<00:00, 628.20it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 628.17it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 628.70it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 627.70it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 628.30it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 626.58it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 628.43it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 627.40it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 628.32it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 627.88it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 629.48it/s]
Evaluating using naive strategy: 4it [41:49, 650.54s/it]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 590/590 [00:00<00:00, 630.47it/s]

testing: 100%|██████████| 590/590 [00:00<00:00, 631.49it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 630.39it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 630.38it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 627.31it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 630.11it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 629.28it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 629.27it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 629.40it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 631.57it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 627.79it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 628.15it/s]
Evaluating using naive strategy: 5it [49:39, 595.93s/it]

Lifelong PR-AUC: 0.4759803647973265, BWT: 0.21407887749450372, FWT: 0.679633574763101


In [25]:
R_replay = evaluation_protocol(T, E, Y, ICL(epochs=100), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 312/312 [00:00<00:00, 666.51it/s]

testing: 100%|██████████| 312/312 [00:00<00:00, 668.76it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 666.25it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 668.62it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 666.99it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 670.12it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 669.97it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 671.03it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 668.28it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 669.40it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 669.06it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 669.85it/s]
Evaluating using replay strategy: 1it [03:57, 237.14s/it]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1550/1550 [00:02<00:00, 663.38it/s]

testing: 100%|██████████| 1550/1550 [00:02<00:00, 668.99it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 665.82it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 668.03it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 668.62it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 666.64it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 668.86it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 670.18it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 667.53it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 668.91it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 666.34it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 667.55it/s]
Evaluating using replay strategy: 2it [23:14, 778.41s/it]

...

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1046/1046 [00:01<00:00, 669.09it/s]

testing: 100%|██████████| 1046/1046 [00:01<00:00, 670.85it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 669.79it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 670.80it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 668.37it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 669.22it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 667.13it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 670.94it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 669.05it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 668.48it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 670.34it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 669.46it/s]
Evaluating using replay strategy: 4it [42:02, 659.18s/it]

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=48, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(49, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 668/668 [00:00<00:00, 670.82it/s]

testing: 100%|██████████| 668/668 [00:00<00:00, 671.63it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 669.70it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 669.75it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 669.65it/s]

testing: 100%|██████████| 731/731 [00:01<00:00, 670.93it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 670.35it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 671.95it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 669.04it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 669.35it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 668.22it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 669.65it/s]
Evaluating using replay strategy: 5it [50:19, 603.90s/it]

Average Inference Time per Evaluation: 0.0000480988 seconds
Lifelong PR-AUC: 0.39922401499880816, BWT: 0.11964614686756707, FWT: 0.5010199439302738


# RCA

In [26]:
from deepod.models.tabular import RCA

In [12]:
R_hm = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 1118.33it/s]

100%|██████████| 10/10 [00:04<00:00,  2.21it/s]

100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

100%|██████████| 10/10 [00:03<00:00,  2.87it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]
Evaluating using hierarchical strategy: 1it [00:26, 26.88s/it]

drift: 0.15575845828560167
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100,


100%|██████████| 10/10 [00:00<00:00, 1088.92it/s]

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]

100%|██████████| 10/10 [00:03<00:00,  2.90it/s]

100%|██████████| 10/10 [00:04<00:00,  2.10it/s]

100%|██████████| 10/10 [00:04<00:00,  2.09it/s]
Evaluating using hierarchical strategy: 2it [00:52, 25.85s/it]

...

drift: 0.08550491766410032
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100,


100%|██████████| 10/10 [00:00<00:00, 1034.23it/s]

100%|██████████| 10/10 [00:04<00:00,  2.23it/s]

100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

100%|██████████| 10/10 [00:03<00:00,  2.87it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]
Evaluating using hierarchical strategy: 4it [01:41, 25.27s/it]

drift: 0.14433198880791898
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100,


100%|██████████| 10/10 [00:00<00:00, 993.84it/s]

100%|██████████| 10/10 [00:04<00:00,  2.22it/s]

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]

100%|██████████| 10/10 [00:03<00:00,  2.88it/s]

100%|██████████| 10/10 [00:04<00:00,  2.07it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]
Evaluating using hierarchical strategy: 5it [02:06, 25.32s/it]

Lifelong ROC-AUC: 0.46783162610323026, BWT: 0.013601554641246794, FWT: 0.35486222628635444


In [20]:
R_ssf = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 14.74it/s]

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]

100%|██████████| 10/10 [00:07<00:00,  1.25it/s]

100%|██████████| 10/10 [00:03<00:00,  2.92it/s]

100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 274.96it/s]


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 14.73it/s]

100%|██████████| 10/10 [00:04<00:00,  2.26it/s]

100%|██████████| 10/10 [00:06<00:00,  1.61it/s]

100%|██████████| 10/10 [00:03<00:00,  2.93it/s]

100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 301.79it/s]


...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 14.69it/s]

100%|██████████| 10/10 [00:04<00:00,  2.26it/s]

100%|██████████| 10/10 [00:06<00:00,  1.61it/s]

100%|██████████| 10/10 [00:03<00:00,  2.93it/s]

100%|██████████| 10/10 [00:04<00:00,  2.12it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 312.10it/s]


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 14.83it/s]

100%|██████████| 10/10 [00:04<00:00,  2.27it/s]

100%|██████████| 10/10 [00:07<00:00,  1.28it/s]

100%|██████████| 10/10 [00:03<00:00,  2.93it/s]

100%|██████████| 10/10 [00:04<00:00,  2.12it/s]

100%|██████████| 10/10 [00:04<00:00,  2.11it/s]
Evaluating using SSF strategy: 5it [02:21, 28.26s/it]

Lifelong ROC-AUC: 0.19839448098273754, BWT: 0.00014011255011197598, FWT: 0.2979311747222437


In [25]:
R_naive = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:02<00:00,  3.44it/s]

100%|██████████| 10/10 [00:04<00:00,  2.01it/s]

100%|██████████| 10/10 [00:06<00:00,  1.46it/s]

100%|██████████| 10/10 [00:03<00:00,  2.66it/s]

100%|██████████| 10/10 [00:05<00:00,  1.92it/s]

100%|██████████| 10/10 [00:05<00:00,  1.92it/s]
Evaluating using naive strategy: 1it [00:45, 45.76s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:13<00:00,  1.37s/it]

100%|██████████| 10/10 [00:04<00:00,  2.04it/s]

100%|██████████| 10/10 [00:06<00:00,  1.46it/s]

100%|██████████| 10/10 [00:03<00:00,  2.66it/s]

100%|██████████| 10/10 [00:05<00:00,  1.93it/s]

100%|██████████| 10/10 [00:05<00:00,  1.93it/s]
Evaluating using naive strategy: 2it [02:29, 79.73s/it]

...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:09<00:00,  1.06it/s]

100%|██████████| 10/10 [00:04<00:00,  2.07it/s]

100%|██████████| 10/10 [00:07<00:00,  1.40it/s]

100%|██████████| 10/10 [00:03<00:00,  2.70it/s]

100%|██████████| 10/10 [00:05<00:00,  1.86it/s]

100%|██████████| 10/10 [00:05<00:00,  1.84it/s]
Evaluating using naive strategy: 4it [04:35, 70.59s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:05<00:00,  1.80it/s]

100%|██████████| 10/10 [00:04<00:00,  2.02it/s]

100%|██████████| 10/10 [00:06<00:00,  1.47it/s]

100%|██████████| 10/10 [00:03<00:00,  2.66it/s]

100%|██████████| 10/10 [00:05<00:00,  1.93it/s]

100%|██████████| 10/10 [00:05<00:00,  1.93it/s]
Evaluating using naive strategy: 5it [05:33, 66.67s/it]

Lifelong PR-AUC: 0.458677884288386, BWT: 0.190154328996587, FWT: 0.4224282129189828


In [27]:
R_replay = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:02<00:00,  3.71it/s]

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]

100%|██████████| 10/10 [00:06<00:00,  1.60it/s]

100%|██████████| 10/10 [00:03<00:00,  2.92it/s]

100%|██████████| 10/10 [00:04<00:00,  2.10it/s]

100%|██████████| 10/10 [00:04<00:00,  2.10it/s]
Evaluating using replay strategy: 1it [00:38, 38.45s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:13<00:00,  1.35s/it]

100%|██████████| 10/10 [00:04<00:00,  2.23it/s]

100%|██████████| 10/10 [00:06<00:00,  1.57it/s]

100%|██████████| 10/10 [00:03<00:00,  2.91it/s]

100%|██████████| 10/10 [00:04<00:00,  2.09it/s]

100%|██████████| 10/10 [00:04<00:00,  2.09it/s]
Evaluating using replay strategy: 2it [02:15, 72.70s/it]

...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:09<00:00,  1.11it/s]

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]

100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

100%|██████████| 10/10 [00:03<00:00,  2.91it/s]

100%|██████████| 10/10 [00:04<00:00,  2.09it/s]

100%|██████████| 10/10 [00:04<00:00,  2.09it/s]
Evaluating using replay strategy: 4it [04:13, 65.37s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=58, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:05<00:00,  1.73it/s]

100%|██████████| 10/10 [00:04<00:00,  2.23it/s]

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]

100%|██████████| 10/10 [00:03<00:00,  2.91it/s]

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]

100%|██████████| 10/10 [00:04<00:00,  2.09it/s]
Evaluating using replay strategy: 5it [05:09, 61.88s/it]

Average Inference Time per Evaluation: 0.0001351466 seconds
Lifelong PR-AUC: 0.37285924558567873, BWT: 0.060578728324494666, FWT: 0.39602854143028715


# RDP

In [28]:
from deepod.models.tabular import RDP

In [14]:
R_hm = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.6s
epoch 10, training loss: 0.000006, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 785.60it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1085.37it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1085.02it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1088.63it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1088.51it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1088.48it/s]
Evaluating using hierarchical strategy: 1it [00:03,  3.67s/it]

drift: 0.15575845828560167
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000010, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 777.44it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1088.93it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1090.84it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1092.39it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1086.89it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1088.63it/s]
Evaluating using hierarchical strategy: 2it [00:07,  3.75s/it]

...

drift: 0.08550491766410032
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000010, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 748.58it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1084.68it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1089.72it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1088.81it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1085.24it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1088.77it/s]
Evaluating using hierarchical strategy: 4it [00:14,  3.54s/it]

drift: 0.14433198880791898
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000009, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 746.85it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1087.08it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1090.08it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1088.44it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1090.00it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1086.60it/s]
Evaluating using hierarchical strategy: 5it [00:17,  3.53s/it]

Lifelong ROC-AUC: 0.3522134732574979, BWT: 0.042147218832008115, FWT: 0.3412602382945243


In [24]:
R_ssf = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000004, time: 0.9s
epoch 10, training loss: 0.000013, time: 0.9s
Start Inference on the training data...



testing: 100%|██████████| 79/79 [00:00<00:00, 1141.60it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1124.20it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1120.68it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1126.22it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1114.33it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 284.55it/s]


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000004, time: 1.0s
epoch 10, training loss: 0.000014, time: 1.0s
Start Inference on the training data...



testing: 100%|██████████| 79/79 [00:00<00:00, 1138.51it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1124.55it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1122.77it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1127.92it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1133.11it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 292.02it/s]


...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000004, time: 0.2s
epoch 10, training loss: 0.000015, time: 0.2s
Start Inference on the training data...



testing: 100%|██████████| 79/79 [00:00<00:00, 1118.74it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1113.77it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1124.80it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1136.02it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1139.04it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 309.79it/s]


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000004, time: 0.2s
epoch 10, training loss: 0.000014, time: 0.2s
Start Inference on the training data...



testing: 100%|██████████| 79/79 [00:00<00:00, 1141.62it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1131.13it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1138.55it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1129.10it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1126.36it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1126.04it/s]
Evaluating using SSF strategy: 5it [00:45,  9.10s/it]

Lifelong ROC-AUC: 0.22051690821185324, BWT: -0.16105950839576202, FWT: 0.48067775760587406


In [28]:
R_naive = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000008, time: 5.1s
epoch 10, training loss: 0.000014, time: 4.5s
Start Inference on the training data...



testing: 100%|██████████| 312/312 [00:00<00:00, 1012.42it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 854.21it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1014.31it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1009.66it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1016.67it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 891.27it/s] 
Evaluating using naive strategy: 1it [00:48, 48.47s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000011, time: 21.0s
epoch 10, training loss: 0.000013, time: 20.9s
Start Inference on the training data...



testing: 100%|██████████| 1472/1472 [00:01<00:00, 904.15it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1015.34it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 885.43it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1018.44it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 850.16it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1014.03it/s]
Evaluating using naive strategy: 2it [04:24, 147.10s/it]

...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000011, time: 13.2s
epoch 10, training loss: 0.000013, time: 14.3s
Start Inference on the training data...



testing: 100%|██████████| 968/968 [00:00<00:00, 1006.65it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1017.93it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 990.11it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1021.98it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1022.06it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1015.42it/s]
Evaluating using naive strategy: 4it [07:46, 121.20s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000009, time: 8.7s
epoch 10, training loss: 0.000012, time: 8.4s
Start Inference on the training data...



testing: 100%|██████████| 590/590 [00:00<00:00, 1010.10it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1017.26it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 979.67it/s] 

testing: 100%|██████████| 401/401 [00:00<00:00, 1023.16it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 978.87it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1021.11it/s]
Evaluating using naive strategy: 5it [09:16, 111.24s/it]

Lifelong PR-AUC: 0.47709673629284577, BWT: 0.23318726648307506, FWT: 0.5024503301170068


In [29]:
R_replay = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000008, time: 0.9s
epoch 10, training loss: 0.000014, time: 0.9s
Start Inference on the training data...



testing: 100%|██████████| 312/312 [00:00<00:00, 1074.37it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1078.52it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1078.61it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1077.69it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1077.76it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1078.21it/s]
Evaluating using replay strategy: 1it [00:12, 12.64s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000013, time: 4.6s
epoch 10, training loss: 0.000015, time: 4.6s
Start Inference on the training data...



testing: 100%|██████████| 1550/1550 [00:01<00:00, 1078.17it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1077.35it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1078.05it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1082.17it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1082.87it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1081.32it/s]
Evaluating using replay strategy: 2it [01:05, 36.02s/it]

...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000011, time: 3.8s
epoch 10, training loss: 0.000013, time: 3.0s
Start Inference on the training data...



testing: 100%|██████████| 1046/1046 [00:01<00:00, 962.92it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1084.55it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1079.24it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1080.18it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1082.41it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1074.33it/s]
Evaluating using replay strategy: 4it [01:59, 31.16s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=58, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000013, time: 2.7s
epoch 10, training loss: 0.000018, time: 2.2s
Start Inference on the training data...



testing: 100%|██████████| 668/668 [00:00<00:00, 1068.59it/s]

testing: 100%|██████████| 520/520 [00:00<00:00, 1077.00it/s]

testing: 100%|██████████| 731/731 [00:00<00:00, 1072.74it/s]

testing: 100%|██████████| 401/401 [00:00<00:00, 1071.93it/s]

testing: 100%|██████████| 555/555 [00:00<00:00, 1070.48it/s]

testing: 100%|██████████| 556/556 [00:00<00:00, 1065.74it/s]
Evaluating using replay strategy: 5it [02:24, 28.86s/it]

Average Inference Time per Evaluation: 0.0000151288 seconds
Lifelong PR-AUC: 0.3634034199662845, BWT: 0.06243627900950712, FWT: 0.3210049019414214
